In [1]:
import utils.dir_utils as dir_utils
import utils.dict_utils as dict_utils
import utils.ptr_utils as ptr_utils
import utils.constants as constants 
import utils.profit as profit 
import helpers.official as official
import utils.csv_utils as csv_utils
import statistics

_, input_df = dir_utils.get_data(combined=True)

In [2]:
def setup():
    # (t[constants.TDATE], official.get_name(t), t[constants.TICKER])
    purchases = set()
    
    for _,t in input_df.iterrows():
        if ptr_utils.isvalid(t[constants.TICKER]):
            core =  (t[constants.TDATE], official.get_name(t), t[constants.TICKER])
            
            if t[constants.TYPE] == 'Purchase' and ptr_utils.get_year(t[constants.TDATE]) <= 2020: 
                purchases.add(core)

    return purchases

p = setup()
print(len(p))

problem_tickers = {}

6162


In [7]:
def get_purch_price(p):
    # 30 days == 4.28  ~~~~~~~ 45 days == 6.42  ~~~~~~~~~~~ 7. 
    buy = [
        ("b_same_day", 0), 
        ("b_seven_weeks", 7),
        ("b_year", 52)
        ]
    
    # (t[constants.TDATE], official.get_name(t), t[constants.TICKER], tdate_purch_edited, tdate_purch_edited_price)
    p_prime = set()
    
    i = 0 

    for (tdate_purch, name_purch, ticker_purch)  in p:
        print(i, tdate_purch, name_purch, ticker_purch)
        for _, increment_buy in buy: 
            if ticker_purch not in problem_tickers: 
                tdate_purch_edited = profit.add_to_date(tdate_purch, increment_buy)
                purch_price = profit.get_stock_price(ticker_purch, tdate_purch_edited)
                if not purch_price:
                    problem_tickers[ticker_purch] = ""
                else: 
                    p_prime.add((tdate_purch, name_purch, ticker_purch, tdate_purch_edited, purch_price))
        i += 1 
    return p_prime
        

p_prime = get_purch_price(p)

0 2020/10/28 Frankel, Lois AXP
1 2020/05/20 Frankel, Lois SO
2 2018/12/31 Perdue Jr., David A. CZR
3 2017/07/20 Roberts, Pat FB
4 2020/04/22 Phillips, Dean PHG
5 2016/02/05 Perdue Jr., David A. ORCL
6 2020/11/27 McCaul, Michael T. FTNT
7 2020/11/27 Cisneros, Gilbert V
8 2020/01/13 Gianforte, Greg CHGCY
9 2020/04/02 Phillips, Dean SRE
10 2015/08/27 Perdue Jr., David A. GLW
11 2019/10/30 Perdue Jr., David A. TCP
12 2020/07/28 Miller, Carol D. AMGN
13 2019/04/16 Shalala, Donna WYND
14 2018/05/04 Perdue Jr., David A. TWX
15 2019/02/15 Shalala, Donna FLR
16 2020/01/29 Cisneros, Gilbert CENT
17 2020/06/05 Lee, Susie NMIH
18 2019/06/25 Perdue Jr., David A. OKE
19 2020/04/14 Carper, Thomas R. PNQI
20 2020/07/09 Carper, Thomas R. C
21 2020/02/28 Phillips, Dean L
22 2017/09/08 Capito, Shelley M. RBGLY
23 2020/02/10 Perdue Jr., David A. HBI
24 2020/05/18 Phillips, Dean UL
25 2020/07/27 Hern, Kevin HD
26 2019/03/18 Shalala, Donna LUKOY
27 2020/03/26 Cisneros, Gilbert SHW
28 2020/03/02 Evans, Dwigh

In [ ]:
# just do a bit of guesswork with intervals with a subset !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# devote more effort to those who are more active !!!!!

def match(buy_increment, problem_tickers):
    # 6 months == 26 == short?
    # 1 year == ~long
    # 2 years == long
    sell = [26, 52, 104]

    res = []
    i = 0 
    for (tdate_purch, name_purch, ticker_purch)  in p:
        if ticker_purch not in problem_tickers: 
            add = [name_purch, ticker_purch, tdate_purch] 
            if buy_increment == 0:
                add.append(tdate_purch)
            tdate_purch_edited = profit.add_to_date(tdate_purch, buy_increment)
            
            for increment_sale in sell: 
                tdate_sale = profit.add_to_date(tdate_purch_edited, increment_sale)
                sale, purch, x = profit.share_diff(ticker_purch, tdate_sale, tdate_purch_edited)
                print(sale, purch, x)
                if x is not None: 
                    if tdate_purch_edited not in add: 
                        add.append(tdate_purch_edited)
                    add.append(tdate_sale)
                    add.append(x)
                else:
                    problem_tickers[ticker_purch] = ""
                    break 
            
            if ticker_purch not in problem_tickers: 
                res.append(add)
                
            # if res != []:
            #     break 

            print(i)
            i += 1 
        
    return res

def get_average(res):
    l = {"0. one_week" : [], 
         "1. two_weeks" : [], 
         "2. four_weeks" : [],
         "3. eight_weeks" : [],
         "4. sixteen_weeks" : [], 
         "5. thirty_two_weeks" : [],
         "6. year" : []}
    
    for [name_purch, ticker_purch, tdate_purch_og, tdate_purch_edited,
         sd_1, p_1, 
         sd_2, p_2,  
         sd_4, p_4, 
         sd_8, p_8,  
         sd_16, p_16,  
         sd_32, p_32,  
         sd_52, p_52] in res:
        
        l["0. one_week"].append(round(p_1,4))
        l["1. two_weeks"].append(round(p_2, 4))
        l["2. four_weeks"].append(round(p_4, 4))
        l["3. eight_weeks"].append(round(p_8, 4))
        l["4. sixteen_weeks"].append(round(p_16, 4))
        l["5. thirty_two_weeks"].append(round(p_32, 4))
        l["6. year"].append(round(p_52, 4))


    for k,v in l.items():
        l[k] = statistics.mean(v)
    return l 

def get_people_average(res):
    # person avg(p1), avg(p2)
    people = {}
    for [name_purch, ticker_purch, tdate_purch_og, tdate_purch_edited,
         sd_1, p_1, 
         sd_2, p_2,  
         sd_4, p_4, 
         sd_8, p_8,  
         sd_16, p_16,  
         sd_32, p_32,  
         sd_52, p_52] in res:
        
        people = dict_utils.increment_list_in_inner_dictionary(people, name_purch, "0. one_week", p_1)
        people = dict_utils.increment_list_in_inner_dictionary(people, name_purch, "1. two_weeks",p_2)
        people = dict_utils.increment_list_in_inner_dictionary(people, name_purch, "2. four_weeks", p_4)
        people = dict_utils.increment_list_in_inner_dictionary(people, name_purch, "3. eight_weeks", p_8)
        people = dict_utils.increment_list_in_inner_dictionary(people, name_purch, "4. sixteen_weeks", p_16)
        people = dict_utils.increment_list_in_inner_dictionary(people, name_purch, "5. thirty_two_weeks", p_32)
        people = dict_utils.increment_list_in_inner_dictionary(people, name_purch, "6. year", p_52)

    finalized = []
    
    for person, d in people.items():
        r = [person]
        for v in d:
            r.append(round(statistics.mean(d[v]), 4))
            
        finalized.append(r)
            
    
    return finalized 

def get_ticker_average(res):
    people = {}
    for [name_purch, ticker_purch, tdate_purch_og, tdate_purch_edited,
         sd_1, p_1, 
         sd_2, p_2,  
         sd_4, p_4, 
         sd_8, p_8,  
         sd_16, p_16,  
         sd_32, p_32,  
         sd_52, p_52] in res:
        
        people = dict_utils.increment_list_in_inner_dictionary(people, ticker_purch, "0. one_week", p_1)
        people = dict_utils.increment_list_in_inner_dictionary(people, ticker_purch, "1. two_weeks",p_2)
        people = dict_utils.increment_list_in_inner_dictionary(people, ticker_purch, "2. four_weeks", p_4)
        people = dict_utils.increment_list_in_inner_dictionary(people, ticker_purch, "3. eight_weeks", p_8)
        people = dict_utils.increment_list_in_inner_dictionary(people, ticker_purch, "4. sixteen_weeks", p_16)
        people = dict_utils.increment_list_in_inner_dictionary(people, ticker_purch, "5. thirty_two_weeks", p_32)
        people = dict_utils.increment_list_in_inner_dictionary(people, ticker_purch, "6. year", p_52)

    finalized = []
    
    for person, d in people.items():
        r = [person]
        for v in d:
            r.append(round(statistics.mean(d[v]), 4))
            
        finalized.append(r)
            
    
    return finalized 


def make_stuff(res, file_buy_increment):
    dir = dir_utils.makesubdir(constants.path_csv, constants.PROFIT+"/"+file_buy_increment)
    

    csv_utils.make_csv_base(dir, file_buy_increment, 
                            [constants.OFFICIAL, constants.TICKER, "tdate_purch_og", "tdate_purch_edited", 
                              '0. one_week', 'profit one_week',  '1. two_weeks', 'profit two_weeks',   '2. four_weeks', 'profit four_weeks',  '3. eight_weeks', 'profit eight_weeks',  '4. sixteen_weeks', 'profit sixteen_weeks',  '5. thirty_two_weeks', 'profit thirty_two_weeks',  '6. year', 'profit year',], res)
   

    get_average_res = get_average(res)
    csv_utils.make_csv(dir, file_buy_increment+"_get_average_",  get_average_res, ["", "Average"])

    get_people_average_res = get_people_average(res)
    csv_utils.make_csv_base(dir, file_buy_increment+"_get_people_average_", [constants.OFFICIAL,  "0. one_week", "1. two_weeks", "2. four_weeks", "3. eight_weeks", "4. sixteen_weeks", "5. thirty_two_weeks", "6. year",], get_people_average_res)

    get_ticker_average_res = get_ticker_average(res)
    csv_utils.make_csv_base(dir, file_buy_increment+"_get_ticker_average_", [constants.TICKER,  "0. one_week", "1. two_weeks", "2. four_weeks", "3. eight_weeks", "4. sixteen_weeks", "5. thirty_two_weeks", "6. year"], get_ticker_average_res)
    
    
# # 38 hours for one iteration !!!!!
# for file_buy_increment, buy_increment in buy: 
#     one_week, problem_tickers = match(buy_increment, problem_tickers)
#     make_stuff(one_week, file_buy_increment)